In [1]:
pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 11.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Prompt 1 - Claude opus 4.1

In [4]:
import os
import base64
import mimetypes
from pathlib import Path
from anthropic import Anthropic

# =========================
# CONFIGURAÇÃO
# =========================
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "COLOQUE AQUI SUA CHAVE")
IMAGE_PATH = "coloque aqui o caminho da imagem"
MODEL_NAME = "claude-opus-4-1-20250805"

# =========================
# PROMPT (sem forçar parse, só pedindo JSON)
# =========================
PROMPT = (
     "Analise a imagem contendo um Diagrama Entidade-Relacionamento (ER) e converta-a para um esquema lógico relacional em formato JSON estruturado.\n\n"
        "Instruções:\n"
        "Para cada entidade, gere uma entrada com:\n\n"
        "- \"attributes\": lista com todos os atributos da tabela\n"
        "- \"pk\": chave primária como lista (mesmo que contenha apenas um atributo)\n"
        "- \"fk\": lista de chaves estrangeiras (se existirem)\n\n"
        "Caso não existam chaves estrangeiras, omita o campo \"fk\".\n"
        "Use nomes significativos, mantendo consistência com o diagrama original.\n\n"
        "O modelo de saída deve seguir rigorosamente o seguinte formato:\n"
        "{\n"
        "  \"Entities\": {\n"
        "    \"EntityName\": {\n"
        "      \"attributes\": [\"x\", \"y\", \"z\"],\n"
        "      \"pk\": [\"a\"],\n"
        "      \"fk\": [\"b\", \"c\"]\n"
        "    },\n"
        "    \"AnotherEntity\": {\n"
        "      \"attributes\": [\"m\", \"n\"],\n"
        "      \"pk\": [\"m\"]\n"
        "    }\n"
        "  }\n"
        "}"
)

# =========================
# FUNÇÃO AUXILIAR
# =========================
def encode_image(image_path: str):
    p = Path(image_path)
    mime, _ = mimetypes.guess_type(p.name)
    if mime is None:
        mime = "image/png"
    with open(p, "rb") as f:
        data_b64 = base64.b64encode(f.read()).decode("utf-8")
    return data_b64, mime

# =========================
# EXECUÇÃO
# =========================
def main():
    client = Anthropic(api_key=ANTHROPIC_API_KEY)
    img_b64, mime = encode_image(IMAGE_PATH)

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0,
        system="Você é um especialista em bancos de dados relacionais. Retorne a saída em JSON no formato pedido.",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": PROMPT},
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": mime,
                            "data": img_b64,
                        },
                    },
                ],
            }
        ],
    )

    # Concatenar resposta de texto
    output = "".join([c.text for c in response.content if c.type == "text"])
    print("\n=== Saída do Claude ===\n")
    print(output)

if __name__ == "__main__":
    main()



=== Saída do Claude ===

Analisando o diagrama ER fornecido, vou converter para o esquema lógico relacional:

```json
{
  "Entities": {
    "EMPLOYEE": {
      "attributes": ["Ssn", "Fname", "Minit", "Lname", "Bdate", "Name", "Address", "Salary", "Sex", "Super_ssn", "Dno"],
      "pk": ["Ssn"],
      "fk": ["Super_ssn", "Dno"]
    },
    "DEPARTMENT": {
      "attributes": ["Number", "Name", "Locations", "Mgr_ssn", "Start_date"],
      "pk": ["Number"],
      "fk": ["Mgr_ssn"]
    },
    "PROJECT": {
      "attributes": ["Number", "Name", "Location", "Controlling_dept"],
      "pk": ["Number"],
      "fk": ["Controlling_dept"]
    },
    "DEPENDENT": {
      "attributes": ["Essn", "Name", "Sex", "Birth_date", "Relationship"],
      "pk": ["Essn", "Name"],
      "fk": ["Essn"]
    },
    "WORKS_ON": {
      "attributes": ["Essn", "Pno", "Hours"],
      "pk": ["Essn", "Pno"],
      "fk": ["Essn", "Pno"]
    }
  }
}
```


## Prompt 2 - Claude opus 4.1

In [2]:
import os
import base64
import mimetypes
from pathlib import Path
from anthropic import Anthropic

# =========================
# CONFIGURAÇÃO
# =========================
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "COLOQUE AQUI SUA CHAVE")
IMAGE_PATH = "coloque aqui o caminho da imagem"
MODEL_NAME = "claude-opus-4-1-20250805"

# =========================
# PROMPT (sem forçar parse, só pedindo JSON)
# =========================
PROMPT = (
    "Tarefa: Converta o Diagrama Entidade-Relacionamento (ER) em um esquema lógico relacional representado em formato JSON.\n\n"
    "Instruções:\n"
    "Para cada entidade ou relacionamento, gere uma entrada com:\n\n"
    "- \"attributes\": lista com todos os atributos da tabela\n"
    "- \"pk\": chave primária como lista (mesmo que contenha apenas um atributo)\n"
    "- \"fk\": lista de chaves estrangeiras (se existirem)\n\n"
    "Caso não existam chaves estrangeiras, omita o campo \"fk\".\n"
    "Use nomes significativos, mantendo consistência com o diagrama original.\n\n"
    "Utilize as regras abaixo para decisão por tabela própria, adição de coluna ou fusão de tabelas:\n\n"
    "Relacionamentos 1:1\n"
    "1. (0,1) — (0,1)\n"
    "Tabela própria: ± (pode ser usada)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "2. (0,1) — (1,1)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ± (pode ser usada)\n"
    "Fusão de tabelas: ✔ (preferida)\n\n"
    "3. (1,1) — (1,1)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ± (pode ser usada)\n"
    "Fusão de tabelas: ✔ (preferida)\n\n"
    "Relacionamentos 1:n\n"
    "1. (0,1) — (0,n)\n"
    "Tabela própria: ± (pode ser usada)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "2. (0,1) — (1,n)\n"
    "Tabela própria: ± (pode ser usada)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "3. (1,1) — (0,n)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "4. (1,1) — (1,n)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "Relacionamentos n:n\n"
    "1. (0,n) — (0,n)\n"
    "Tabela própria: ✔ (preferida)\n"
    "Adição de coluna: ✘ (não usar)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "2. (0,n) — (1,n)\n"
    "Tabela própria: ✔ (preferida)\n"
    "Adição de coluna: ✘ (não usar)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "3. (1,n) — (1,n)\n"
    "Tabela própria: ✔ (preferida)\n"
    "Adição de coluna: ✘ (não usar)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "O modelo de saída deve seguir rigorosamente o seguinte formato:\n"
    "{\n"
    "  \"Entities\": {\n"
    "    \"EntityName\": {\n"
    "      \"attributes\": [\"x\", \"y\", \"z\"],\n"
    "      \"pk\": [\"a\"],\n"
    "      \"fk\": [\"b\", \"c\"]\n"
    "    },\n"
    "    \"AnotherEntity\": {\n"
    "      \"attributes\": [\"m\", \"n\"],\n"
    "      \"pk\": [\"m\"]\n"
    "    }\n"
    "  }\n"
    "}"
)

# =========================
# FUNÇÃO AUXILIAR
# =========================
def encode_image(image_path: str):
    p = Path(image_path)
    mime, _ = mimetypes.guess_type(p.name)
    if mime is None:
        mime = "image/png"
    with open(p, "rb") as f:
        data_b64 = base64.b64encode(f.read()).decode("utf-8")
    return data_b64, mime

# =========================
# EXECUÇÃO
# =========================
def main():
    client = Anthropic(api_key=ANTHROPIC_API_KEY)
    img_b64, mime = encode_image(IMAGE_PATH)

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0,
        system="Você é um especialista em bancos de dados relacionais. Retorne a saída em JSON no formato pedido.",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": PROMPT},
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": mime,
                            "data": img_b64,
                        },
                    },
                ],
            }
        ],
    )

    # Concatenar resposta de texto
    output = "".join([c.text for c in response.content if c.type == "text"])
    print("\n=== Saída do Claude ===\n")
    print(output)

if __name__ == "__main__":
    main()



=== Saída do Claude ===

Analisando o diagrama ER fornecido, vou converter para o esquema relacional seguindo as regras especificadas:

```json
{
  "Entities": {
    "EMPLOYEE": {
      "attributes": ["Ssn", "Fname", "Minit", "Lname", "Bdate", "Name", "Address", "Salary", "Sex", "Dno", "Super_ssn"],
      "pk": ["Ssn"],
      "fk": ["Dno", "Super_ssn"]
    },
    "DEPARTMENT": {
      "attributes": ["Number", "Name", "Locations", "Mgr_ssn", "Mgr_start_date"],
      "pk": ["Number"],
      "fk": ["Mgr_ssn"]
    },
    "PROJECT": {
      "attributes": ["Number", "Name", "Location", "Controlling_dept"],
      "pk": ["Number"],
      "fk": ["Controlling_dept"]
    },
    "DEPENDENT": {
      "attributes": ["Employee_ssn", "Name", "Sex", "Birth_date", "Relationship"],
      "pk": ["Employee_ssn", "Name"],
      "fk": ["Employee_ssn"]
    },
    "WORKS_ON": {
      "attributes": ["Employee_ssn", "Project_number", "Hours"],
      "pk": ["Employee_ssn", "Project_number"],
      "fk": ["Employ